In [81]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import copy

font_path = "C:/Windows/Fonts/MALGUN.TTF"
font_name = fm.FontProperties(fname=font_path).get_name()
plt.rc('font', family=font_name)
plt.rc('xtick', labelsize=10)
mpl.rcParams['axes.unicode_minus'] = False

In [82]:
X_train_df_sel = pd.read_csv("../data/dacon_parking_demand_dataset/preprocessed_data/X_train_df.csv")
X_test_df_sel = pd.read_csv("../data/dacon_parking_demand_dataset/preprocessed_data/X_test_df.csv")
y_train_df = pd.read_csv("../data/dacon_parking_demand_dataset/preprocessed_data/y_train_df.csv")
sub = pd.read_csv("../data/dacon_parking_demand_dataset/test.csv")
age = pd.read_csv("../data/dacon_parking_demand_dataset/age_gender_info.csv")

In [83]:
X_train_ori = X_train_df_sel.copy(deep=True)
X_test_ori = X_test_df_sel.copy(deep=True)

In [84]:
X_train_df_sel = X_train_df_sel.drop(["단지코드"], axis=1)
X_test_df_sel = X_test_df_sel.drop(["단지코드"], axis=1)

In [85]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train_df_sel)

X_train_df_scaled = scaler.transform(X_train_df_sel)
X_test_df_scaled = scaler.transform(X_test_df_sel)
X_test_df_scaled[:5]

array([[-0.24854825,  0.09980247,  0.1015166 , -0.07385873, -0.40235963,
        -0.6388798 , -1.23153021, -0.08586964, -0.1850583 , -0.49738446,
        -0.11430211, -0.03235359, -0.05609682,  4.18445665, -0.13181763,
        -0.19297121, -0.10787079, -0.10787079, -0.02641199, -0.10279641,
        -0.01867284, -0.25095121,  0.21128301, -0.26405319,  1.95495492,
        -0.38059474, -0.20061544, -0.22734603, -0.19199747, -0.31967136,
        -0.35928128, -0.0918469 , -0.10951231, -0.23238849, -0.19872784,
        -0.18099   , -0.24141474, -0.24543128, -0.49356501,  0.49356501,
        -0.01793852,  0.12848285],
       [-0.24854825, -0.54553172,  0.1015166 ,  0.66893512, -0.40235963,
        -0.6388798 ,  0.81199794, -0.08586964, -0.1850583 , -0.49738446,
        -0.11430211, -0.03235359, -0.05609682, -0.23897965, -0.13181763,
        -0.19297121, -0.10787079, -0.10787079, -0.02641199, -0.10279641,
        -0.01867284, -0.25095121, -0.55080313, -0.26405319,  1.95495492,
        -0.38059

### 랜덤포레스트회귀 모델 102.80211

In [86]:
# 최고 스코어 모델 -> 피처 42개 적용 및 모델 디폴트 설정으로 학습

from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_jobs=-1, random_state=42)
model.fit(X_train_df_scaled, y_train_df)
pred = model.predict(X_test_df_scaled)

result = pd.DataFrame(data=pred, columns=["전용면적 당 차량등록수"], index=X_test_ori.index)
result["단지코드"] = X_test_ori.loc[X_test_ori.index, "단지코드"]

pred_group = result.groupby("단지코드")["전용면적 당 차량등록수"].sum()
final_pred = pd.Series(pred_group)
final_pred = final_pred.reset_index(drop=False)

sub_df = sub.merge(final_pred, how="left")
sub_df = sub_df.drop_duplicates(["단지코드"], keep="first").reset_index()
sub_df = sub_df[["단지코드", "전용면적 당 차량등록수"]]
sub_df = sub_df.rename(columns={"단지코드":"code", "전용면적 당 차량등록수": "num"})
# sub_df.to_csv("rf_regressor_with_전용면적_5의배수_전처리_0720.csv", index=False)

sub_df

<ipython-input-86-806d379ceabe>:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train_df_scaled, y_train_df)


,code,num
0,C1072,778.888696
1,C1128,1346.914529
2,C1456,551.346741
3,C1840,474.976179
4,C1332,1211.934694
...,...,...
145,C2456,261.030966
146,C1266,381.882374
147,C2152,53.162481
148,C1267,392.868991
